<a href="https://colab.research.google.com/github/root-epifit/madmo-adv/blob/my_exercise/text-generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Char-based text generation with LSTM

In [1]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [31]:
TRAIN_TEXT_FILE_PATH = 'train_text.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'н', '\n', 'т', 'и', 'л', 'с', 'р', 'в', 'д', 'м', ',', 'у', 'к', 'ь', 'ы', 'й', 'п', 'я', 'г', 'б', 'з', 'ч', 'ж', 'х', '.', 'ш', 'ю', 'И', ';', 'Н', 'X', 'В', 'I', 'П', 'О', 'ц', 'Т', 'С', 'К', ':', '!', 'Д', '?', 'щ', 'Б', 'Е', 'V', 'М', 'У', '…', 'Ч', 'З', 'Л', 'э', 'Г', '«', '»', '–', 'Я', '-', 'А', 'ё', '—', 'L', 'Р', 'ф', 'e', 'Х', ')', '(', 't', 'a', 'ъ', 'Ж', 'l', 'o', 'r', 'n', 'Ш', 'i', 'Ф', 's', 'u', 'd', 'm', 'Ц', 'b', 'c', 'Э', 'v', 'M', '€', 'N', 'A', 'f', 'S', '7', 'Ю', 'h', 'B', 'P', 'p', '[', ']', '’', 'T', 'k', 'z', 'g', '^', 'y', '5', 'C', 'H', 'Y', 'Q', 'q', 'W', 'R', 'G', 'D', '8', '9']


In [33]:
sequence[:20]

array([36,  5,  0,  5,  0,  0,  0, 59, 51,  1, 19,  0, 12, 21, 12, 21,  0,
        9,  3, 13])

In [3]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [4]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [5]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [35]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 5000            # 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        #print(predicted_text)

Loss: 3.0130217981338503
  оне слолоте,
   Оа но век овей вол вел велов но стово волом но волель ситет сеть вело слево воте но вел но но сет во сет вен вут осно но вот оне сей но та витоге бата  отел вето но воги но селом ен в
Loss: 2.3970568132400514
 Он седень,
   Но на додоль нони в поровий нась,
   И в не не в в вал накодоль стого достать оненье тора столь оми сторы столь
   На тополь столь сторень вавасто не поренья сторей
   В порего порать по
Loss: 2.212592353820801
 Прадать додной веренье
   И сториль сердей,
   И в порать стом он пововет,
   Ей не поволь вать вась вас вавной,
   Не поволь сторой сенет,
   Не пороченевит поли,
   И полкой поленье мора
   Дорил не
Loss: 2.0953247213363646
 Пороженный не не своей не подой
   И на столь в подором он он он сторей
   Покой на в породать со в не стольной
   Постом пора не годой нам,
   И породно столить сторой
   Но постала постовь сельный,

Loss: 2.0030565643310547
  И столете не на нам
   И год он совенья моей.
   И стольно в на сл

In [38]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='.'
    )
)

.
   Старик одной не свой поэта,
   Которой постеле нас него
   С ней девочки с первый верной
   В потом и пора на последней,
   И девы молодой души,
   В приятный толковой воскресенье
   И в семьей полукруг верный,
   И сердце своего знал
   В сумрак не всё был поэта,
   Онегин в письмо правда ли?),
   В постеле в сердце не преданья,
   Не полу забыть молчать слышали,
   Не под толковать поэта страстей
   И совет и не верный себе
   Не видит на старины нет.
   Он волненье не приветы,
   И встретились вот она всем,
   С ней судьбу старой верной вольных,
   Обеды молодой волшебный
   Под сон уж нет соседи привычной,
   И подруг не верно пора,
   Под старый постеле свой пете
   С какой первый приветы света
   С полный на пред ним и приводит
   Страшно не наших друг с пете,
   Не предо не верно не свет
   И кровь верный друг содревий,
   Не под ним она с ней поэта,
   С ней постеле на полку свой,
   Не преданья в семьей раздела
   И полно не полно простой,
   В семьей под ним и кончалась,

In [50]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=256, 
    start_text='Татьяна'
    )
)

Татьяна сладо страстей ума
   Простите с ней нет уста нет
   И с ней обедом приводит,
   В обращаясь он не знал
   Простите мне с ней порой страданье
   И тайны так в первый старин,
   Но полный мой все для всех доле
   И в семьей развел ведали
   И с нею подость
